# 提升方法 Boosting

Kearns和Valiant在概率近似正确PAC（probably approximately correct）学习的框架中：  
强可学习（strongly learnable),一个概念，如果存在一个多项式的学习算法能够学习它，并且正确率很高，那么称这个概念是强可学习的；  
弱可学习（weakly learnable),一个概念，如果存在一个多项式的学习算法能够学习它，并且正确率仅比随机猜测略好，那么称这个概念是弱可学习的；  
Schapire证明强可学习与弱可学习是等价的。



输入：训练数据集$T=\{(x_1,y_1),(x_2,y_2),\ldots,(x_N,y_N)\}$，其中$x_i \in R^n , y_i \in \{-1,+1\}$，弱学习算法  
输出：最终分类器G(x)

(1)初始化训练数据的权值分布  
$D_1=(w_11,w_12,\ldots,w_1N), w_{1i}=\frac{1}{N}$

(2)对$m=1,2,\ldots,M$  
(a)使用具有权值分布D_m的训练数据集学习，得到基本分类器
$$
G_m(x):\mathcal{x} \rightarrow \{-1,+1\}
$$


(b)计算$G_m(x)$在训练数据集上的分类误差
$$
e_m=P(G_m(x_i) \ne y_i)=\sum_{i=1}^N w_{mi} I(G(x_i) \ne y_i)
$$

(c)计算$G_m(x)$的系数
$$
\alpha_m=\frac{1}{2}\log{\frac{1-e_m}{e_m}}
$$
这里的对数是自然对数

(d)更新训练数据集的权值分布
$$
D_{m+1}=(w_{m+1,1},\ldots,w_{m+1,N}) \\
w_{m+1,i}=\frac{w_{mi}}{Z_m} \exp(-\alpha_m y_i G_m(x_i))
$$
这里，$Z_m$是规范化因子
$$
Z_m=\sum_{i=1}^N w_{mi} \exp(-\alpha_m y_i G_m(x_i))
$$

(3)构建基本分类器的线性组合
$$
f(x)=\sum_{m=1}^M \alpha_m G_m(x)\\
G(x)=sign(f(x))
$$

## AdaBoost 算法的训练误差分析
AdaBoost算法最终分类器的训练误差界为
$$
\frac 1 N \sum_{i=1}^N I(G(x_i) \ne y_i) \le \frac{1}{N} \sum_{i=1}^N \exp(-y_i f(x_i))=\prod_m Z_m
$$

二类分类问题AdaBoost的训练误差边界
$$
\prod_{m=1}^M  Z_m=\prod _{m=1}^M[2\sqrt{e_m(1-e_m)}]=\prod_{m=1}^M\sqrt{1-4  {\gamma_m}^2}=\exp(-2\sum_{m=1}^M {\gamma_m}^2)
$$

证明
$$
\begin{eqnarray}
Z_m &=& \sum_{i=1}^N w_{mi} \exp(-\alpha_m y_i G_m(x_i)) \\
&=& \sum_{y_i G_m(x_i)=1} w_{mi} \exp(-\alpha_m)+\sum_{y_i G_m(x_i)=-1} w_{mi} \exp(\alpha_m) \\
&=& \sqrt{\frac{e_m}{1-e_m}}\sum_{y_i G_m(x_i)=1} w_{mi}+\sqrt{\frac{1-e_m}{e_m}} \sum_{y_i G_m(x_i)=-1} w_{mi} \\
&=& 2\sqrt{e_m(1-e_m)} \\
&=& \sqrt{1-4 {\gamma_m}^2}
\end{eqnarray}
$$
其中$\gamma_m=\frac 1 2 -e_m$

如果存在$\gamma \gt 0$,对于所有$m,\gamma_m \ge \gamma$，那么
$$
\frac 1 N \sum_{i=1}^N I(G(x_i) \ne y_i)  \le \exp(-2\sum_{m=1}^M {\gamma_m}^2) \le \exp(-2M{\gamma}^2)
$$

可以看到AdaBoost的训练误差是以指数速率下降的。这里的Ada是Adaptive适应的意思，即它能自适应弱分类器各自的训练误差率

## AdaBoost算法的解释


### 前向分步算法 foward stagewise algorithm
加法模型（additvie model）
$$
f(x)=\sum_{m=1}^M \beta_m b(x;\gamma_m)
$$

损失函数最小化：
$$
\begin{eqnarray}
\min_{\beta_m,\gamma_m} \sum_{i=1}^N L(y_i,\sum_{m=1}^M \beta_m b(x_i;\gamma_m))
\end{eqnarray}
$$

### 前向分步算法与AdaBoost
前向分步算法的损失函数是损失函数（exponential loss function）
$$
L(y,f(x))=\exp[-yf(x)]
$$

假设经过m-1轮迭代前向分步算法已经得到$f_{m-1}(x)$

$$
f_{m-1}(x)=f_{m-2}(x)+\ldots+\alpha_{m-1}G_{m-1}(x)=\sum_{i=1}^{m-1} \alpha_i G_i(x)
$$

每一次目标
$$
(\alpha_m,G_m(x))=\arg \min_{\alpha,G} \sum_{i=1}^N \exp(-y_i (f_{m-1}(x_i)+\alpha G(x_i)))
$$

$$
(\alpha_m,G_m(x))=\arg \min_{\alpha,G} \sum_{i=1}^N \bar{w}_{mi} \exp(-y_i \alpha G(x_i))
$$

分两步，第一步求$G_m^*(x)$：
$$
G_m^*(x)=\arg \min \sum_{i=1}^N \bar{w}_{mi} I(y_i \ne G(x_i))
$$

第二步：
$$
\begin{eqnarray}
\sum_{i=1}^N \bar{w}_{mi} I(y_i \ne G(x_i)) &=& \sum_{y_i=G_m(x_i)}\bar{w}_{mi} e^{-\alpha}+\sum_{y_i \ne G_m(x_i)}\bar{w}_{mi}e^\alpha \\
&=& (e^\alpha -e^{-\alpha})\sum_{i=1}^N\bar{w}_{mi}I(y \ne G(x_i))+e^{-\alpha}\sum_{i=1}^N \bar{w}_{mi}
\end{eqnarray}
$$

对$\alpha$求导得
$$
\alpha_m^*=\frac{1}{2}\log{\frac{1-e_m}{e_m}}
$$

## 提升树
### 提升树模型
提升方法实际采用加法模型（即基函数的线性组合）与前向分步算法。以决策树为基函数的提升方法称为提升树（decision tree）

J个分类的树可以表示为
$$
T(x,\Theta)=\sum_{j=1}^JI(x \in R_j)
$$
其中T是基函数，$\Theta$是参数，$R_j$是区域

$$
L(y,f(x))=(y-f(x))^2 
$$

$$
\begin{eqnarray}
L(y,f_{m-1}(x)+T(x;\Theta) &=& (y-f_{m-1}(x)-T(x;\Theta))^2 \\
&=& (r-T(x;\Theta))^2
\end{eqnarray}
$$

这里$r=y-f_{m-1}(x)$是当前模型拟合数据的残差，对回归问题的提升算法来说，只需要简单的拟合当前模型的残差

$$
f_M(x)=\sum_{m=1}^M T(x;\Theta)
$$

### 梯度提升(略)